# Training Genome Selection Strategy

We use **27 curated reference genomes** selected for maximum diversity and quality.

## 1. Taxonomic Diversity
- **Proteobacteria** (8 genomes): Gram-negative, diverse metabolism
- **Firmicutes** (7 genomes): Gram-positive, low GC
- **Actinobacteria** (4 genomes): High GC content
- **Archaea** (5 genomes): Distinct codon usage
- **Mixed** (3 genomes): Edge cases, extreme GC

## 2. Genome Characteristics
- **GC content range**: 25% - 75%
- **Genome size**: 1.5 Mbp - 10 Mbp
- **Gene density**: 85% - 95%
- **Codon usage**: Diverse patterns

## 3. Annotation Quality
- All from **NCBI RefSeq** (highest quality)
- **Complete genomes** (not drafts)
- Well-studied model organisms where possible
- Manual curation verified

## 4. Practical Considerations
- Represent common human/environmental pathogens
- Include extremophiles for edge cases
- Balance between model organisms and diversity

---

### Why Curated > Random?
 **Reproducibility**: Same training set = same model  
 **Balance**: Prevents taxonomic bias  
 **Quality**: High-confidence annotations  
 **Interpretability**: Can analyze per-group performance  
 **Standard Practice**: Matches industry tools (Prodigal, GeneMark)

---

**Reference:** Similar approach used by Prodigal (Hyatt et al. 2010)

In [1]:
import numpy as np
import torch
import pickle
from sklearn.metrics import f1_score
from pathlib import Path

# Taxonomic groups
GROUP_TAXIDS = {
    "Proteobacteria": 1224,
    "Firmicutes": 1239,
    "Actinobacteria": 201174,
    "Archaea": 2157,
}

# Curated training genomes with metadata
TRAINING_GENOMES = {
    # Proteobacteria (Gram-negative) - 8 genomes
    'NC_011751.1': {'name': 'Escherichia coli', 'gc': 0.51, 'size': 4.6},
    'NC_017626.1': {'name': 'Salmonella enterica', 'gc': 0.52, 'size': 4.8},
    'NC_009004.1': {'name': 'Pseudomonas aeruginosa', 'gc': 0.66, 'size': 6.3},
    'NC_007912.1': {'name': 'Helicobacter pylori', 'gc': 0.39, 'size': 1.6},
    'NC_002946.2': {'name': 'Neisseria meningitidis', 'gc': 0.52, 'size': 2.2},
    'NC_004547.2': {'name': 'Vibrio cholerae', 'gc': 0.48, 'size': 4.0},
    'NC_007005.1': {'name': 'Xanthomonas campestris', 'gc': 0.65, 'size': 5.1},
    'NC_009656.1': {'name': 'Acinetobacter baumannii', 'gc': 0.39, 'size': 3.9},
    
    # Firmicutes (Gram-positive, low GC) - 7 genomes
    'NC_014479.1': {'name': 'Bacillus subtilis', 'gc': 0.43, 'size': 4.2},
    'NC_020244.1': {'name': 'Staphylococcus aureus', 'gc': 0.33, 'size': 2.8},
    'NC_003212.1': {'name': 'Listeria monocytogenes', 'gc': 0.38, 'size': 2.9},
    'NC_002570.2': {'name': 'Clostridium perfringens', 'gc': 0.29, 'size': 3.0},
    'NC_007795.1': {'name': 'Streptococcus pyogenes', 'gc': 0.38, 'size': 1.9},
    'NC_004461.1': {'name': 'Lactococcus lactis', 'gc': 0.35, 'size': 2.4},
    'NC_002737.2': {'name': 'Enterococcus faecalis', 'gc': 0.37, 'size': 3.2},
    
    # Actinobacteria (High GC) - 4 genomes
    'NC_008595.1': {'name': 'Mycobacterium tuberculosis', 'gc': 0.66, 'size': 4.4},
    'NC_002755.2': {'name': 'Mycobacterium leprae', 'gc': 0.58, 'size': 3.3},
    'NC_009338.1': {'name': 'Corynebacterium diphtheriae', 'gc': 0.53, 'size': 2.5},
    'NC_014363.1': {'name': 'Streptomyces coelicolor', 'gc': 0.72, 'size': 8.7},
    
    # High GC / Edge cases - 3 genomes
    'NC_003888.3': {'name': 'Streptomyces avermitilis', 'gc': 0.71, 'size': 9.0},
    'NC_009921.1': {'name': 'Frankia sp.', 'gc': 0.73, 'size': 7.5},
    'NC_008148.1': {'name': 'Bifidobacterium longum', 'gc': 0.60, 'size': 2.5},
    
    # Archaea - 5 genomes
    'NC_013967.1': {'name': 'Haloarcula marismortui', 'gc': 0.62, 'size': 4.3},
    'NC_015666.1': {'name': 'Methanococcus maripaludis', 'gc': 0.33, 'size': 1.7},
    'NC_007796.1': {'name': 'Methanospirillum hungatei', 'gc': 0.45, 'size': 3.5},
    'NC_013790.1': {'name': 'Methanobrevibacter smithii', 'gc': 0.31, 'size': 1.9},
    'NC_000909.1': {'name': 'Methanocaldococcus jannaschii', 'gc': 0.31, 'size': 1.7},
}

print(f"Total training genomes: {len(TRAINING_GENOMES)}")
print(f"GC range: {min(g['gc'] for g in TRAINING_GENOMES.values()):.2f} - "
      f"{max(g['gc'] for g in TRAINING_GENOMES.values()):.2f}")
print(f"Size range: {min(g['size'] for g in TRAINING_GENOMES.values()):.1f} - "
      f"{max(g['size'] for g in TRAINING_GENOMES.values()):.1f} Mbp")

Total training genomes: 27
GC range: 0.29 - 0.73
Size range: 1.6 - 9.0 Mbp


# Hybrid Deep Learning Model Variants

This notebook will systematically test multiple hybrid DL architectures to find the optimal configuration for gene prediction refinement.

---

## Model Architectures

All models use the same **hybrid architecture**:
- **CNN Branch**: Learns sequence patterns from raw DNA (1500bp one-hot encoded)
- **Dense Branch**: Uses 31 traditional features (codon, IMM, RBS, length, etc.)
- **Fusion Layer**: Combines both branches for final classification

---

## Experimental Variants

We will train and compare the following configurations:

### **Variant 1: Binary Final Filter**
- **When Applied**: After step 7 (same as Variant 1)
- **Input**: ~5,000 final gene predictions per genome
- **Output Classes**: 
  - `0`: FALSE_POSITIVE
  - `1`: REAL_GENE
- **Filtering Strategy**: Keep if probability > threshold
- **Goal**: Simple binary decision for precision


### **Variant 4: Binary Initial Filter**
- **When Applied**: After step 3 (same as Variant 3)
- **Input**: ~50,000 ORFs per genome
- **Output Classes**: Same as Variant 2
- **Filtering Strategy**: Keep if probability > threshold
- **Goal**: Simple early filtering

---

## Comparison Matrix

| Variant | Model Type | Position | Input Size | Classes | Flexibility |
|---------|-----------|----------|------------|---------|-------------|
| **1** | Binary | Final | ~5k | 2 | Low (single threshold) |
| **2** | Binary | Initial | ~50k | 2 | Low |

---

## Success Metrics

For each variant, we will measure on **validation genomes**:

- **Sensitivity (Recall)**: TP / (TP + FN) - Don't miss real genes
- **Precision**: TP / (TP + FP) - Minimize false positives
- **F1 Score**: Harmonic mean of sensitivity and precision
- **Number of Predictions**: Final gene count
- **Computational Cost**: Training time, inference time

**Target Performance:**
- Baseline (no hybrid DL): Sens ~75%, Prec ~65%, F1 ~69%
- Goal: F1 > 75% (at least +6% improvement)

---
**Conservative (High Precision):**
```python

In [ ]:
# file: src/test_training_genome.py

from pathlib import Path
import sys

# --- Setup imports ---
sys.path.insert(0, str(Path(__file__).resolve().parent.parent))  # add parent dir

from hybrid_predictor import predict_ncbi_genome
from src.config import TRAINING_GENOMES


def inspect_prediction_features(predictions: list, genome_id: str):
    """Inspect and report available features in first prediction."""
    if not predictions:
        print("\n[!] No predictions found - check pipeline!")
        return

    first_pred = predictions[0]

    required_base = [
        'sequence',  # For CNN
        'codon_score_norm', 'imm_score_norm', 'rbs_score_norm',
        'length_score_norm', 'start_score_norm', 'combined_score'
    ]

    required_additional = [
        'gc_content', 'gc_skew', 'at_skew', 'purine_content',
        'codon_adaptation_index', 'effective_num_codons', 'codon_bias_index',
        'length_bp', 'length_codons', 'length_log',
        'start_codon_type', 'has_kozak_like', 'stop_codon_type', 
        'has_hairpin_near_stop',
        'hydrophobicity_mean', 'hydrophobicity_std', 'charge_mean',
        'aromatic_fraction', 'small_fraction', 'polar_fraction',
        'overlap_with_others', 'distance_to_nearest', 'gene_density_region',
        'is_nested', 'num_alternative_starts'
    ]

    has_base = [f for f in required_base if f in first_pred]
    has_additional = [f for f in required_additional if f in first_pred]
    missing = [f for f in required_additional if f not in first_pred]

    print(f"\n{'='*80}")
    print(f"FEATURE INSPECTION FOR: {genome_id}")
    print(f"{'='*80}")
    print(f"✓ Base features: {len(has_base)}/{len(required_base)}")
    print(f"✓ Additional features: {len(has_additional)}/{len(required_additional)}")
    print(f"⚠ Missing features: {len(missing)}")

    if missing:
        print("\nMissing feature names:")
        for feat in missing:
            print(f"  - {feat}")
        print(f"\n→ Next: Use TraditionalFeatureExtractor to add missing features")
    else:
        print("\n✓ All required features are present! Ready for training data generation.")


def run_training_genome_test(email: str):
    """Download one training genome from NCBI and run full hybrid predictor pipeline."""
    print("=" * 80)
    print("TEST PIPELINE ON SINGLE TRAINING GENOME")
    print("=" * 80)

    genome_id = next(iter(TRAINING_GENOMES))
    genome_info = TRAINING_GENOMES[genome_id]

    print(f"\nGenome: {genome_id}")
    print(f"Organism: {genome_info['name']}")
    print(f"GC%: {genome_info['gc']}")
    print(f"Size: {genome_info['size']} Mbp")

    # --- Run hybrid predictor directly ---
    predictions = predict_ncbi_genome(accession=genome_id, email=email)

    # --- Inspect result ---
    inspect_prediction_features(predictions, genome_id)
    return predictions

if __name__ == "__main__":
   final_predictions= run_training_genome_test(email="your.email@example.com")


TEST PIPELINE ON SINGLE TRAINING GENOME

Test genome: NC_011751.1
Organism: Escherichia coli
GC%: 0.51
Size: 4.6 Mbp

DOWNLOADING FROM NCBI
Accession: NC_011751.1
Email: your.email@example.com

  Fetching sequence from NCBI...
  Downloaded: 5,350,781 bytes
  ✓ Download successful!


Loading sequence from: c:\Users\User\Desktop\Bacterial Gene Prediction & Comparison Project\data\full_dataset\NC_011751.1.fasta
✓ Loaded: 5,202,090 bp

RUNNING PIPELINE: NC_011751.1

STEP 1: FIND ORFs
Detecting ORFs and calculating RBS...
Complete: 196,142 ORFs detected with RBS scores
✓ Found 196,142 ORFs

STEP 2: CREATE TRAINING SET
✓ Training set: 1,243 ORFs

STEP 3: CREATE INTERGENIC SET
✓ Intergenic regions: 533

STEP 4: BUILD SCORING MODELS
Building traditional scoring models...
  Building codon usage models...
  Building IMM models...
✓ All models built in 3.5s
  IMM order: 7
  Training sequences: 1243 (1,936,119 bp)
  Intergenic sequences: 533 (168,432 bp)
✓ Models built

STEP 5: SCORE ALL ORFs
Scor

In [7]:
# Simple feature inspection

print("="*80)
print("FEATURES IN FINAL PREDICTIONS")
print("="*80)

if final_predictions:
    first_pred = final_predictions[0]
    
    print(f"\nTotal fields: {len(first_pred)}")
    print(f"\nAll fields in prediction:")
    
    for key in sorted(first_pred.keys()):
        value = first_pred[key]
        
        if key == 'sequence':
            print(f"  {key:30s} : (DNA sequence, {len(value)} bp)")
        elif isinstance(value, float):
            print(f"  {key:30s} : {value:.4f}")
        elif isinstance(value, int):
            print(f"  {key:30s} : {value}")
        elif isinstance(value, str):
            print(f"  {key:30s} : '{value}'")
        else:
            print(f"  {key:30s} : {value}")
    
    print(f"\n{'='*80}")

else:
    print("\n[!] No predictions available")

FEATURES IN FINAL PREDICTIONS

Total fields: 24

All fields in prediction:
  codon_score                    : 0.2934
  codon_score_norm               : 1.0018
  combined_score                 : 6.4343
  end                            : 2757
  frame                          : 0
  genome_end                     : 2757
  genome_start                   : 295
  imm_score                      : 0.1919
  imm_score_norm                 : 0.8986
  length                         : 2463
  length_score                   : 2.1054
  length_score_norm              : 2.3526
  rbs_motif                      : 'AGGAGG'
  rbs_score                      : 13.8000
  rbs_score_norm                 : 1.0463
  rbs_sequence                   : 'ACGAGGTA'
  rbs_spacing                    : 7
  sequence                       : (DNA sequence, 2463 bp)
  start                          : 295
  start_codon                    : 'ATG'
  start_score                    : 1.0000
  start_score_norm               : 1.1350


In [55]:
# Feature extraction - DE NOVO COMPATIBLE ONLY

import math
import numpy as np
from collections import Counter
from Bio.Seq import Seq
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from typing import Dict, List

def extract_all_features(predictions):
    """
    Add 19 missing features (DE NOVO COMPATIBLE ONLY).
    
    REMOVED:
    - CAI (requires reference genes)
    - Gene context features (requires knowing real genes)
    
    Total: 25 features (not 26)
    """
    print(f"\n{'='*80}")
    print(f"EXTRACTING DE NOVO FEATURES FOR {len(predictions):,} PREDICTIONS")
    print(f"{'='*80}")
    
    for i, pred in enumerate(predictions):
        if i % 1000 == 0 and i > 0:
            print(f"  {i:,}...")
        
        sequence = pred['sequence']
        seq_upper = sequence.upper()
        
        # =====================================================================
        # EASY DERIVATIONS (9 features)
        # =====================================================================
        
        # Length features (3)
        pred['length_bp'] = float(pred['length'])
        pred['length_codons'] = float(pred['length'] / 3)
        pred['length_log'] = math.log(max(pred['length'], 1))
        
        # Start/stop codon types (2)
        start_map = {'ATG': 0, 'GTG': 1, 'TTG': 2}
        pred['start_codon_type'] = float(start_map.get(pred['start_codon'], 0))
        
        stop_codon = seq_upper[-3:]
        stop_map = {'TAA': 0, 'TAG': 1, 'TGA': 2}
        pred['stop_codon_type'] = float(stop_map.get(stop_codon, 0))
        
        # Kozak-like (1)
        pred['has_kozak_like'] = float(pred.get('rbs_score', 0) > 3.0)
        
        # Sequence composition (3)
        counts = Counter(seq_upper)
        length = len(seq_upper)
        
        g = counts.get('G', 0)
        c = counts.get('C', 0)
        a = counts.get('A', 0)
        t = counts.get('T', 0)
        
        pred['gc_content'] = (g + c) / length if length > 0 else 0.0
        pred['gc_skew'] = (g - c) / (g + c) if (g + c) > 0 else 0.0
        pred['at_skew'] = (a - t) / (a + t) if (a + t) > 0 else 0.0
        
        # =====================================================================
        # COMPLEX CALCULATIONS (10 features) - ALL DE NOVO COMPATIBLE
        # =====================================================================
        
        # Purine content (1)
        pred['purine_content'] = (a + g) / length if length > 0 else 0.0
        
        # Codon metrics (2) - REMOVED CAI!
        pred['effective_num_codons'] = calculate_enc(seq_upper)
        pred['codon_bias_index'] = calculate_cbi(seq_upper)
        
        # Hairpin near stop (1)
        pred['has_hairpin_near_stop'] = detect_hairpin_near_stop(seq_upper)
        
        # Amino acid properties (6)
        aa_props = calculate_amino_acid_properties(seq_upper)
        pred['hydrophobicity_mean'] = aa_props['hydro_mean']
        pred['hydrophobicity_std'] = aa_props['hydro_std']
        pred['charge_mean'] = aa_props['charge_mean']
        pred['aromatic_fraction'] = aa_props['aromatic_frac']
        pred['small_fraction'] = aa_props['small_frac']
        pred['polar_fraction'] = aa_props['polar_frac']
    
    print(f"✓ Feature extraction complete")
    return predictions


def calculate_enc(sequence):
    """
    Effective Number of Codons (de novo - uses only this sequence).
    
    Measures codon usage diversity within this ORF.
    """
    codons = [sequence[i:i+3] for i in range(0, len(sequence)-2, 3)]
    valid_codons = [c for c in codons if len(c) == 3 and 'N' not in c]
    
    if len(valid_codons) == 0:
        return 0.0
    
    codon_counts = Counter(valid_codons)
    num_unique = len(codon_counts)
    
    # Normalized ENC: unique codons / total codons
    # Range: 0 (all same codon) to ~1 (very diverse)
    enc_normalized = num_unique / max(len(valid_codons), 1)
    
    return float(enc_normalized)


def calculate_cbi(sequence):
    """
    Codon Bias Index.
    
    Entropy-based measure of codon usage evenness.
    """
    codons = [sequence[i:i+3] for i in range(0, len(sequence)-2, 3)]
    valid_codons = [c for c in codons if len(c) == 3 and 'N' not in c]
    
    if len(valid_codons) == 0:
        return 0.0
    
    codon_counts = Counter(valid_codons)
    frequencies = np.array(list(codon_counts.values())) / len(valid_codons)
    
    # Shannon entropy
    entropy = -np.sum(frequencies * np.log2(frequencies + 1e-10))
    max_entropy = math.log2(61)  # 61 sense codons
    
    # CBI = 1 - normalized entropy
    # High CBI = biased (few codons used), Low CBI = uniform (many codons)
    cbi = 1.0 - (entropy / max_entropy) if max_entropy > 0 else 0.0
    
    return float(cbi)


def detect_hairpin_near_stop(sequence, window=30):
    """
    Detect potential hairpin structure near stop codon.
    
    De novo - only uses this sequence.
    """
    if len(sequence) < window:
        return 0.0
    
    tail = sequence[-window:]
    
    try:
        seq_obj = Seq(tail)
        reverse_comp = str(seq_obj.reverse_complement())
        
        # Count complementary matches (hairpin indicator)
        matches = sum(1 for a, b in zip(tail, reverse_comp) if a == b)
        hairpin_score = matches / len(tail)
        
        return 1.0 if hairpin_score > 0.6 else 0.0
    except:
        return 0.0


def calculate_amino_acid_properties(sequence):
    """
    Calculate amino acid properties (de novo - only this sequence).
    
    Uses BioPython for translation and analysis.
    """
    try:
        # Translate using bacterial genetic code
        seq_obj = Seq(sequence)
        protein = str(seq_obj.translate(table=11, to_stop=True))
        
        if len(protein) == 0:
            return {
                'hydro_mean': 0.0, 'hydro_std': 0.0, 'charge_mean': 0.0,
                'aromatic_frac': 0.0, 'small_frac': 0.0, 'polar_frac': 0.0
            }
        
        # Use BioPython's ProteinAnalysis
        analysis = ProteinAnalysis(protein)
        aa_percent = analysis.get_amino_acids_percent()
        
        # AA property sets
        aromatic = set('FYW')
        small = set('AGSTCV')
        polar = set('STNQY')
        charged = set('DEKR')
        
        # Calculate fractions
        aromatic_frac = sum(aa_percent.get(aa, 0) for aa in aromatic)
        small_frac = sum(aa_percent.get(aa, 0) for aa in small)
        polar_frac = sum(aa_percent.get(aa, 0) for aa in polar)
        charge_mean = sum(aa_percent.get(aa, 0) for aa in charged)
        
        # Kyte-Doolittle hydrophobicity scale
        kd_scale = {
            'A': 1.8, 'R': -4.5, 'N': -3.5, 'D': -3.5, 'C': 2.5,
            'Q': -3.5, 'E': -3.5, 'G': -0.4, 'H': -3.2, 'I': 4.5,
            'L': 3.8, 'K': -3.9, 'M': 1.9, 'F': 2.8, 'P': -1.6,
            'S': -0.8, 'T': -0.7, 'W': -0.9, 'Y': -1.3, 'V': 4.2
        }
        
        hydro_values = [kd_scale.get(aa, 0.0) for aa in protein]
        
        return {
            'hydro_mean': float(np.mean(hydro_values)) if hydro_values else 0.0,
            'hydro_std': float(np.std(hydro_values)) if len(hydro_values) > 1 else 0.0,
            'charge_mean': float(charge_mean),
            'aromatic_frac': float(aromatic_frac),
            'small_frac': float(small_frac),
            'polar_frac': float(polar_frac),
        }
    
    except Exception as e:
        # Translation failed - return zeros
        return {
            'hydro_mean': 0.0, 'hydro_std': 0.0, 'charge_mean': 0.0,
            'aromatic_frac': 0.0, 'small_frac': 0.0, 'polar_frac': 0.0
        }



In [ ]:

# =============================================================================
# RUN EXTRACTION
# =============================================================================

print("="*80)
print("DE NOVO FEATURE EXTRACTION")
print("="*80)

# Extract features
final_predictions = extract_all_features(final_predictions)

# Verify
print(f"\n{'='*80}")
print(f"VERIFICATION")
print(f"{'='*80}")

first_pred = final_predictions[0]
print(f"\nTotal fields now: {len(first_pred)}")

# Check for new features (19 added)
new_features = [
    'length_bp', 'length_codons', 'length_log',
    'start_codon_type', 'stop_codon_type', 'has_kozak_like',
    'gc_content', 'gc_skew', 'at_skew', 'purine_content',
    'effective_num_codons', 'codon_bias_index', 
    'has_hairpin_near_stop',
    'hydrophobicity_mean', 'hydrophobicity_std', 'charge_mean',
    'aromatic_fraction', 'small_fraction', 'polar_fraction'
]

print(f"\nNew de novo features added ({len(new_features)}):")
for feat in new_features:
    if feat in first_pred:
        value = first_pred[feat]
        print(f"  ✓ {feat:30s} : {value:.4f}")
    else:
        print(f"  ✗ {feat:30s} : MISSING!")

print(f"\n{'='*80}")
print(f"FINAL FEATURE COUNT")
print(f"{'='*80}")
print(f"Existing scores: 6")
print(f"New features: {len(new_features)}")
print(f"Total features: {6 + len(new_features)} (DE NOVO COMPATIBLE)")
print(f"\nFor Hybrid DL:")
print(f"  - CNN branch: 1 input (DNA sequence)")
print(f"  - Dense branch: 25 inputs (traditional features)")
print(f"  - Total: 26 inputs")

print(f"\n{'='*80}")
print(f"✓ READY FOR TRAINING DATA GENERATION")
print(f"{'='*80}")

In [21]:
import sys
from pathlib import Path

# Add parent directory (where hybrid_predictor.py is) to the import path
sys.path.append(str(Path.cwd().parent))
from hybrid_predictor import predict_ncbi_genome
results = {}

for acc in TRAINING_GENOMES:
    preds = predict_ncbi_genome(acc, email="youremail@example.com")
    results[acc] = preds

print(f"\n✓ Completed predictions for {len(results)} genomes.")


MODE: NCBI DOWNLOAD
Accession: NC_011751.1
Email: youremail@example.com

  Target: data\full_dataset\NC_011751.1.fasta
  Fetching sequence...
  Downloaded: 5,350,781 bytes
  ✓ Genome downloaded successfully

RUNNING GENE PREDICTION


MODE: RAW FASTA FILE
Input: data\full_dataset\NC_011751.1.fasta
Output: C:\Users\User\Desktop\Bacterial Gene Prediction & Comparison Project\results\NC_011751.1_predictions.gff
ML filtering: Enabled
ML threshold: 0.1

STEP 1: LOAD FASTA FILE
Loaded: NC_011751.1 Escherichia coli UMN026, complete sequence
Genome size: 5,202,090 bp
Sequence loaded: 5,202,090 bp

STEP 2: FIND ALL ORFs
Detecting ORFs and calculating RBS...
Complete: 196,142 ORFs detected with RBS scores
ORFs found: 196,142

STEP 3: CREATE TRAINING SET
Training set: 1,243 ORFs

STEP 4: CREATE INTERGENIC REGIONS
Intergenic regions: 533

STEP 5: BUILD SCORING MODELS
Building traditional scoring models...
  Building codon usage models...
  Building IMM models...
✓ All models built in 4.0s
  IMM or

Groups user_genome: 100%|██████████| 7770/7770 [00:00<00:00, 9506.07it/s] 
c:\Users\User\anaconda3\envs\gene_prediction\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(



STEP 10: SELECT BEST START CODONS

Selecting best start for 5,804 groups
  Single option groups: 629
  Multiple option groups: 5,175
Top candidates: 5,804

STEP 11: FINAL FILTERING
Filtered: 5,480 kept, 324 removed
Final predictions: 5,480

STEP 12: WRITE OUTPUT
[+] Wrote 5480 predictions to C:\Users\User\Desktop\Bacterial Gene Prediction & Comparison Project\results\NC_011751.1_predictions.gff

PREDICTION COMPLETE!
Input:  data\full_dataset\NC_011751.1.fasta
Output: C:\Users\User\Desktop\Bacterial Gene Prediction & Comparison Project\results\NC_011751.1_predictions.gff
Size:   5,202,090 bp
Genes:  5,480


NCBI PREDICTION COMPLETE!
Genome:      NC_011751.1
Downloaded:  data\full_dataset\NC_011751.1.fasta
Predictions: results\NC_011751.1_predictions.gff
Total genes: 5,480


MODE: NCBI DOWNLOAD
Accession: NC_017626.1
Email: youremail@example.com

  Target: data\full_dataset\NC_017626.1.fasta
  Fetching sequence...
  Downloaded: 5,391,805 bytes
  ✓ Genome downloaded successfully

RUNNING

Groups user_genome: 100%|██████████| 7801/7801 [00:00<00:00, 9503.69it/s]
c:\Users\User\anaconda3\envs\gene_prediction\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(



STEP 10: SELECT BEST START CODONS

Selecting best start for 5,889 groups
  Single option groups: 669
  Multiple option groups: 5,220
Top candidates: 5,889

STEP 11: FINAL FILTERING
Filtered: 5,565 kept, 324 removed
Final predictions: 5,565

STEP 12: WRITE OUTPUT
[+] Wrote 5565 predictions to C:\Users\User\Desktop\Bacterial Gene Prediction & Comparison Project\results\NC_017626.1_predictions.gff

PREDICTION COMPLETE!
Input:  data\full_dataset\NC_017626.1.fasta
Output: C:\Users\User\Desktop\Bacterial Gene Prediction & Comparison Project\results\NC_017626.1_predictions.gff
Size:   5,241,977 bp
Genes:  5,565


NCBI PREDICTION COMPLETE!
Genome:      NC_017626.1
Downloaded:  data\full_dataset\NC_017626.1.fasta
Predictions: results\NC_017626.1_predictions.gff
Total genes: 5,565


MODE: NCBI DOWNLOAD
Accession: NC_009004.1
Email: youremail@example.com

  Target: data\full_dataset\NC_009004.1.fasta
  Fetching sequence...
  Downloaded: 2,601,829 bytes
  ✓ Genome downloaded successfully

RUNNING

...

Groups user_genome: 100%|██████████| 3625/3625 [00:00<00:00, 8384.88it/s]
c:\Users\User\anaconda3\envs\gene_prediction\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(



STEP 10: SELECT BEST START CODONS

Selecting best start for 2,537 groups
  Single option groups: 289
  Multiple option groups: 2,248
Top candidates: 2,537

STEP 11: FINAL FILTERING
Filtered: 2,504 kept, 33 removed
Final predictions: 2,504

STEP 12: WRITE OUTPUT
[+] Wrote 2504 predictions to C:\Users\User\Desktop\Bacterial Gene Prediction & Comparison Project\results\NC_013790.1_predictions.gff

PREDICTION COMPLETE!
Input:  data\full_dataset\NC_013790.1.fasta
Output: C:\Users\User\Desktop\Bacterial Gene Prediction & Comparison Project\results\NC_013790.1_predictions.gff
Size:   2,937,203 bp
Genes:  2,504


NCBI PREDICTION COMPLETE!
Genome:      NC_013790.1
Downloaded:  data\full_dataset\NC_013790.1.fasta
Predictions: results\NC_013790.1_predictions.gff
Total genes: 2,504


MODE: NCBI DOWNLOAD
Accession: NC_000909.1
Email: youremail@example.com

  Target: data\full_dataset\NC_000909.1.fasta
  Fetching sequence...
  Downloaded: 1,712,616 bytes
  ✓ Genome downloaded successfully

RUNNING 

Groups user_genome: 100%|██████████| 1780/1780 [00:00<00:00, 8697.52it/s]


STEP 10: SELECT BEST START CODONS

Selecting best start for 1,587 groups
  Single option groups: 99
  Multiple option groups: 1,488
Top candidates: 1,587

STEP 11: FINAL FILTERING
Filtered: 1,572 kept, 15 removed
Final predictions: 1,572

STEP 12: WRITE OUTPUT
[+] Wrote 1572 predictions to C:\Users\User\Desktop\Bacterial Gene Prediction & Comparison Project\results\NC_000909.1_predictions.gff

PREDICTION COMPLETE!
Input:  data\full_dataset\NC_000909.1.fasta
Output: C:\Users\User\Desktop\Bacterial Gene Prediction & Comparison Project\results\NC_000909.1_predictions.gff
Size:   1,664,970 bp
Genes:  1,572


NCBI PREDICTION COMPLETE!
Genome:      NC_000909.1
Downloaded:  data\full_dataset\NC_000909.1.fasta
Predictions: results\NC_000909.1_predictions.gff
Total genes: 1,572


✓ Completed predictions for 27 genomes.



c:\Users\User\anaconda3\envs\gene_prediction\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [2]:
import pandas as pd
from pathlib import Path
from Bio import Entrez

In [ ]:

def combine_genome_datasets(all_genome_data):
    """
    Combine all genome DataFrames into one large dataset with labels and features.
    Returns:
        combined_df: pandas DataFrame containing all genomes.
    """
    dfs = []
    for gid, df in all_genome_data.items():
        df = df.copy()
        df["genome_id"] = gid  # keep track of which genome each ORF came from
        dfs.append(df)

    combined_df = pd.concat(dfs, ignore_index=True)
    
    print(f"Combined dataset shape: {combined_df.shape}")
    print(f"Number of genomes: {len(all_genome_data)}")
    print(f"Columns: {list(combined_df.columns)}")

    return combined_df

def assign_labels_to_all_results(results: dict):
    """
    Convert all prediction results to DataFrames with binary labels.
    Downloads missing GFFs from NCBI into data/full_dataset under parent folder.
    """
    labeled_results = {}
    
    parent_dir = Path.cwd().parent  # parent folder of current notebook/script
    full_dataset_dir = parent_dir / "data" / "full_dataset"
    full_dataset_dir.mkdir(parents=True, exist_ok=True)
    
    for genome_id, orfs in results.items():
        print(f"\n=== Labeling genome: {genome_id} ===")
        reference_gff = full_dataset_dir / f"{genome_id}.gff"

        # --- Download GFF if missing ---
        if not reference_gff.exists():
            print(f"  Reference GFF not found locally, attempting to download from NCBI...")
            try:
                try:
                    from src.config import NCBI_EMAIL
                    Entrez.email = NCBI_EMAIL
                except ImportError:
                    Entrez.email = "user@example.com"  # fallback
                
                print(f"  Downloading GFF from NCBI...")
                handle = Entrez.efetch(
                    db="nucleotide",
                    id=genome_id,
                    rettype="gff3",
                    retmode="text"
                )
                gff_content = handle.read()
                handle.close()
                
                with open(reference_gff, "w") as f:
                    f.write(gff_content)
                print(f"  Saved GFF to {reference_gff}")
            except Exception as e:
                print(f"⚠ Could not download GFF for {genome_id}: {e}")
                continue

        # --- Convert ORFs to DataFrame ---
        pred_df = pd.DataFrame(orfs)
        if "genome_start" not in pred_df.columns or "genome_end" not in pred_df.columns:
            raise ValueError(f"{genome_id} predictions missing genome_start/genome_end")
        
        # --- Load reference CDS ---
        ref = pd.read_csv(reference_gff, sep="\t", comment="#", header=None)
        ref = ref[ref[2] == "CDS"][[3, 4]].rename(columns={3: "start", 4: "end"}).drop_duplicates()
        
        # --- Assign binary labels ---
        pred_df["label"] = 0
        for _, row in ref.iterrows():
            matches = (pred_df["genome_start"] == row["start"]) & (pred_df["genome_end"] == row["end"])
            pred_df.loc[matches, "label"] = 1
        
        labeled_results[genome_id] = pred_df
        print(f"✓ {genome_id}: {len(pred_df)} ORFs labeled (label=1: {pred_df['label'].sum()})")
    
    print(f"\n✓ All prediction results converted to DataFrames.")
    return labeled_results

for acc, preds in results.items():
    if isinstance(preds, list):
        results[acc] = pd.DataFrame(preds)
    elif not isinstance(preds, pd.DataFrame):
        raise TypeError(f"Unexpected data type for {acc}: {type(preds)}")

labeled_results = assign_labels_to_all_results(results)

print("✓ All prediction results converted to DataFrames.")

combined_df = combine_genome_datasets(labeled_results)



=== Labeling genome: NC_011751.1 ===
✓ NC_011751.1: 5480 ORFs labeled (label=1: 3893)

=== Labeling genome: NC_017626.1 ===
✓ NC_017626.1: 5565 ORFs labeled (label=1: 3866)

=== Labeling genome: NC_009004.1 ===
✓ NC_009004.1: 2308 ORFs labeled (label=1: 1950)

=== Labeling genome: NC_007912.1 ===
✓ NC_007912.1: 3914 ORFs labeled (label=1: 3331)

=== Labeling genome: NC_002946.2 ===
✓ NC_002946.2: 2859 ORFs labeled (label=1: 1402)

=== Labeling genome: NC_004547.2 ===
✓ NC_004547.2: 4927 ORFs labeled (label=1: 3520)

=== Labeling genome: NC_007005.1 ===
✓ NC_007005.1: 8049 ORFs labeled (label=1: 4140)

=== Labeling genome: NC_009656.1 ===
✓ NC_009656.1: 8165 ORFs labeled (label=1: 4660)

=== Labeling genome: NC_014479.1 ===
✓ NC_014479.1: 3695 ORFs labeled (label=1: 2878)

=== Labeling genome: NC_020244.1 ===
✓ NC_020244.1: 3703 ORFs labeled (label=1: 2905)

=== Labeling genome: NC_003212.1 ===
✓ NC_003212.1: 2826 ORFs labeled (label=1: 2460)

=== Labeling genome: NC_002570.2 ===
✓ NC_

In [39]:
import pandas as pd
from pathlib import Path
from Bio import Entrez

In [42]:
# Extract all features for the combined dataset using existing function

print("="*80)
print("FEATURE EXTRACTION FOR COMBINED DATASET")
print("="*80)

# Check current state
print(f"\nCurrent DataFrame:")
print(f"  Shape: {combined_df.shape}")
print(f"  Total predictions: {len(combined_df):,}")
print(f"  Columns: {len(combined_df.columns)}")

# Convert DataFrame to list of dicts
print(f"\nConverting to list format...")
predictions_list = combined_df.to_dict('records')
print(f"  ✓ Converted {len(predictions_list):,} predictions")

# Extract features using the existing function
print(f"\nExtracting features...")
predictions_with_features = extract_all_features(predictions_list)

# Convert back to DataFrame
print(f"\nConverting back to DataFrame...")
combined_df = pd.DataFrame(predictions_with_features)
print(f"  ✓ DataFrame rebuilt")

# Verify all required features are present
print(f"\n{'='*80}")
print(f"VERIFICATION")
print(f"{'='*80}")

required_features = [
    # Core
    'sequence', 'label', 'genome_id',
    # Traditional scores (6)
    'codon_score_norm', 'imm_score_norm', 'rbs_score_norm',
    'length_score_norm', 'start_score_norm', 'combined_score',
    # New features (19)
    'length_bp', 'length_codons', 'length_log',
    'start_codon_type', 'stop_codon_type', 'has_kozak_like',
    'gc_content', 'gc_skew', 'at_skew', 'purine_content',
    'effective_num_codons', 'codon_bias_index',
    'has_hairpin_near_stop',
    'hydrophobicity_mean', 'hydrophobicity_std', 'charge_mean',
    'aromatic_fraction', 'small_fraction', 'polar_fraction'
]

print(f"\nChecking {len(required_features)} required features:")
missing = []
for feat in required_features:
    if feat in combined_df.columns:
        print(f"  ✓ {feat}")
    else:
        print(f"  ✗ {feat} - MISSING!")
        missing.append(feat)

# Label distribution
print(f"\n{'='*80}")
print(f"LABEL DISTRIBUTION")
print(f"{'='*80}")
label_counts = combined_df['label'].value_counts()
for label in sorted(label_counts.index):
    count = label_counts[label]
    pct = count / len(combined_df) * 100
    label_name = "REAL_GENE" if label == 1 else "NOT_PROTEIN"
    print(f"  {label} ({label_name:15s}): {count:6,} ({pct:5.1f}%)")

# Final summary
print(f"\n{'='*80}")
print(f"FINAL SUMMARY")
print(f"{'='*80}")
print(f"Total predictions: {len(combined_df):,}")
print(f"Total genomes: {combined_df['genome_id'].nunique()}")
print(f"Total features: {len(combined_df.columns)}")
print(f"Missing features: {len(missing)}")

if len(missing) == 0:
    print(f"\n✓ ALL FEATURES PRESENT - READY FOR PYTORCH!")
    print(f"\n{'='*80}")
    print(f"NEXT STEP: Create PyTorch Dataset")
    print(f"{'='*80}")
else:
    print(f"\n⚠ Need to fix {len(missing)} missing features")
    print(f"Missing: {missing}")

# Save the complete dataset
output_file = Path.cwd().parent / "data" / "hybrid_dl_training" / "training_data_complete.pkl"
output_file.parent.mkdir(parents=True, exist_ok=True)

print(f"\nSaving complete dataset...")
combined_df.to_pickle(output_file)
print(f"✓ Saved to: {output_file}")
print(f"  Size: {output_file.stat().st_size / 1024 / 1024:.1f} MB")


FEATURE EXTRACTION FOR COMBINED DATASET

Current DataFrame:
  Shape: (126749, 26)
  Total predictions: 126,749
  Columns: 26

Converting to list format...
  ✓ Converted 126,749 predictions

Extracting features...

EXTRACTING DE NOVO FEATURES FOR 126,749 PREDICTIONS


c:\Users\User\anaconda3\envs\gene_prediction\lib\site-packages\Bio\SeqUtils\ProtParam.py:106: BiopythonDeprecationWarning: The get_amino_acids_percent method has been deprecated and will likely be removed from Biopython in the near future. Please use the amino_acids_percent attribute instead.
  warnings.warn(


  1,000...
  2,000...
  3,000...
  4,000...
  5,000...
  6,000...
  7,000...
  8,000...
  9,000...
  10,000...
  11,000...
  12,000...
  13,000...
  14,000...
  15,000...
  16,000...
  17,000...
  18,000...
  19,000...
  20,000...
  21,000...
  22,000...
  23,000...
  24,000...
  25,000...
  26,000...
  27,000...
  28,000...
  29,000...
  30,000...
  31,000...
  32,000...
  33,000...
  34,000...
  35,000...
  36,000...
  37,000...
  38,000...
  39,000...
  40,000...
  41,000...
  42,000...
  43,000...
  44,000...
  45,000...
  46,000...
  47,000...
  48,000...
  49,000...
  50,000...
  51,000...
  52,000...
  53,000...
  54,000...
  55,000...
  56,000...
  57,000...
  58,000...
  59,000...
  60,000...
  61,000...
  62,000...
  63,000...
  64,000...
  65,000...
  66,000...
  67,000...
  68,000...
  69,000...
  70,000...
  71,000...
  72,000...
  73,000...
  74,000...
  75,000...
  76,000...
  77,000...
  78,000...
  79,000...
  80,000...
  81,000...
  82,000...
  83,000...
  84,000...
 

In [8]:
from pathlib import Path
import pandas as pd

# -----------------------------
# 📂 Load the complete dataset
# -----------------------------
input_file = Path.cwd().parent / "data" / "hybrid_dl_training" / "training_data_complete.pkl"

print(f"Loading dataset from: {input_file}")
combined_df = pd.read_pickle(input_file)
print(f"✓ Loaded successfully — shape: {combined_df.shape}")
print(f"  Size: {input_file.stat().st_size / 1024 / 1024:.1f} MB")


Loading dataset from: c:\Users\User\Desktop\Bacterial Gene Prediction & Comparison Project\data\hybrid_dl_training\training_data_complete.pkl
✓ Loaded successfully — shape: (126749, 45)
  Size: 160.0 MB


In [9]:
# CORRECT: Split by ORF, use ALL 27 genomes

import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.model_selection import train_test_split

# =============================================================================
# DATASET CLASSES (from your approach)
# =============================================================================

BASES = 'ACGT'
BASE_TO_IDX = {b: i for i, b in enumerate(BASES)}

def one_hot_encode(seq):
    """One-hot encode DNA sequence."""
    seq = seq.upper()
    arr = np.zeros((len(seq), len(BASES)), dtype=np.float32)
    for i, b in enumerate(seq):
        if b in BASE_TO_IDX:
            arr[i, BASE_TO_IDX[b]] = 1.0
    return arr

class ORFDataset(Dataset):
    """Dataset for variable-length sequences with traditional features."""
    
    def __init__(self, df, traditional_features):
        self.sequences = df['sequence'].tolist()
        self.labels = df['label'].values.astype(np.float32)
        self.trad_feats = df[traditional_features].values.astype(np.float32)
        
        # Analyze genome representation
        if 'genome_id' in df.columns:
            genome_counts = df['genome_id'].value_counts()
            print(f"Dataset: {len(self)} samples from {len(genome_counts)} genomes")
        else:
            print(f"Dataset: {len(self)} samples")
        
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        seq = one_hot_encode(self.sequences[idx])
        trad_feat = self.trad_feats[idx]
        label = self.labels[idx]
        return torch.tensor(seq), torch.tensor(trad_feat), torch.tensor(label)

def collate_fn(batch):
    """Collate with dynamic padding to batch max length."""
    seqs, trad_feats, labels = zip(*batch)
    
    seq_lens = [s.shape[0] for s in seqs]
    max_len = max(seq_lens)
    
    padded_seqs = torch.zeros(len(seqs), max_len, 4)
    for i, s in enumerate(seqs):
        padded_seqs[i, :s.shape[0], :] = s
    
    trad_feats = torch.stack(trad_feats)
    labels = torch.stack(labels)
    
    return padded_seqs, trad_feats, labels


# =============================================================================
# CREATE DATASETS - CORRECT WAY
# =============================================================================

print("="*80)
print("CREATING DATASETS - SPLIT BY ORF (USE ALL 27 GENOMES)")
print("="*80)

traditional_features = [
    'codon_score_norm', 'imm_score_norm', 'rbs_score_norm',
    'length_score_norm', 'start_score_norm', 'combined_score',
    'length_bp', 'length_codons', 'length_log',
    'start_codon_type', 'stop_codon_type', 'has_kozak_like',
    'gc_content', 'gc_skew', 'at_skew', 'purine_content',
    'effective_num_codons', 'codon_bias_index',
    'has_hairpin_near_stop',
    'hydrophobicity_mean', 'hydrophobicity_std', 'charge_mean',
    'aromatic_fraction', 'small_fraction', 'polar_fraction'
]

# Check genome representation
print(f"\nTotal dataset:")
print(f"  Samples: {len(combined_df):,}")
print(f"  Genomes: {combined_df['genome_id'].nunique()}")

genome_dist = combined_df['genome_id'].value_counts()
print(f"\nSamples per genome:")
print(f"  Min: {genome_dist.min():,}")
print(f"  Max: {genome_dist.max():,}")
print(f"  Mean: {genome_dist.mean():.0f}")

# CORRECT: Split by ORF (rows), not by genome
print(f"\n{'='*80}")
print(f"SPLITTING BY ORF (80/20)")
print(f"{'='*80}")

train_df, val_df = train_test_split(
    combined_df,
    test_size=0.2,
    random_state=42,
    stratify=combined_df['label']  # Maintain label balance
)

print(f"\nSplit result:")
print(f"  Training samples: {len(train_df):,} ({len(train_df)/len(combined_df)*100:.1f}%)")
print(f"  Validation samples: {len(val_df):,} ({len(val_df)/len(combined_df)*100:.1f}%)")

# Verify ALL genomes are in both sets
train_genomes = set(train_df['genome_id'].unique())
val_genomes = set(val_df['genome_id'].unique())

print(f"\nGenome representation:")
print(f"  Training: {len(train_genomes)} genomes")
print(f"  Validation: {len(val_genomes)} genomes")
print(f"  Overlap: {len(train_genomes & val_genomes)} genomes (should be all 27!)")

if len(train_genomes & val_genomes) == 27:
    print(f"  ✓ All 27 genomes represented in BOTH train and val!")
else:
    print(f"  ⚠ Warning: Not all genomes in both sets")

# Check label balance
print(f"\nLabel distribution:")

train_labels = train_df['label'].value_counts()
print(f"  Training:")
for label in sorted(train_labels.index):
    count = train_labels[label]
    pct = count / len(train_df) * 100
    label_name = "REAL_GENE" if label == 1 else "NOT_PROTEIN"
    print(f"    {label} ({label_name:15s}): {count:6,} ({pct:5.1f}%)")

val_labels = val_df['label'].value_counts()
print(f"  Validation:")
for label in sorted(val_labels.index):
    count = val_labels[label]
    pct = count / len(val_df) * 100
    label_name = "REAL_GENE" if label == 1 else "NOT_PROTEIN"
    print(f"    {label} ({label_name:15s}): {count:6,} ({pct:5.1f}%)")

# Create PyTorch Datasets
print(f"\n{'='*80}")
print(f"CREATING PYTORCH DATASETS")
print(f"{'='*80}\n")

print("Training Dataset:")
train_dataset = ORFDataset(train_df, traditional_features)

print("\nValidation Dataset:")
val_dataset = ORFDataset(val_df, traditional_features)

# Create DataLoaders
batch_size = 64
train_loader = DataLoader(
    train_dataset, 
    batch_size=batch_size, 
    shuffle=True, 
    collate_fn=collate_fn,
    num_workers=0
)
val_loader = DataLoader(
    val_dataset, 
    batch_size=batch_size, 
    shuffle=False, 
    collate_fn=collate_fn,
    num_workers=0
)

print(f"\n{'='*80}")
print(f"DATALOADERS CREATED")
print(f"{'='*80}")
print(f"  Batch size: {batch_size}")
print(f"  Training batches: {len(train_loader):,}")
print(f"  Validation batches: {len(val_loader):,}")

# Test
print(f"\n{'='*80}")
print(f"TESTING")
print(f"{'='*80}")

for batch_seqs, batch_feats, batch_labels in train_loader:
    print(f"\nBatch shapes:")
    print(f"  Sequences: {batch_seqs.shape}")
    print(f"  Features: {batch_feats.shape}")
    print(f"  Labels: {batch_labels.shape}")
    break

print(f"\n{'='*80}")
print(f"✓ DATASETS READY - CORRECT SPLIT STRATEGY!")
print(f"{'='*80}")
print(f"\nKey points:")
print(f"  ✓ All 27 genomes used for training")
print(f"  ✓ Validation checks for overfitting (20% of ORFs)")
print(f"  ✓ Stratified split maintains label balance")
print(f"  ✓ Later: Test on SEPARATE genomes for true generalization")

CREATING DATASETS - SPLIT BY ORF (USE ALL 27 GENOMES)

Total dataset:
  Samples: 126,749
  Genomes: 27

Samples per genome:
  Min: 1,572
  Max: 10,463
  Mean: 4694

SPLITTING BY ORF (80/20)

Split result:
  Training samples: 101,399 (80.0%)
  Validation samples: 25,350 (20.0%)

Genome representation:
  Training: 27 genomes
  Validation: 27 genomes
  Overlap: 27 genomes (should be all 27!)
  ✓ All 27 genomes represented in BOTH train and val!

Label distribution:
  Training:
    0 (NOT_PROTEIN    ): 41,116 ( 40.5%)
    1 (REAL_GENE      ): 60,283 ( 59.5%)
  Validation:
    0 (NOT_PROTEIN    ): 10,279 ( 40.5%)
    1 (REAL_GENE      ): 15,071 ( 59.5%)

CREATING PYTORCH DATASETS

Training Dataset:
Dataset: 101399 samples from 27 genomes

Validation Dataset:
Dataset: 25350 samples from 27 genomes

DATALOADERS CREATED
  Batch size: 64
  Training batches: 1,585
  Validation batches: 397

TESTING

Batch shapes:
  Sequences: torch.Size([64, 3486, 4])
  Features: torch.Size([64, 25])
  Labels: t

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm

# ========================
# MODEL DEFINITION
# ========================

class CNNBranch(nn.Module):
    def __init__(self, output_dim=128):
        super(CNNBranch, self).__init__()
        self.conv1 = nn.Conv1d(4, 64, 7, padding=3)
        self.bn1 = nn.BatchNorm1d(64)
        self.pool1 = nn.MaxPool1d(2)
        self.conv2 = nn.Conv1d(64, 128, 5, padding=2)
        self.bn2 = nn.BatchNorm1d(128)
        self.pool2 = nn.MaxPool1d(2)
        self.conv3 = nn.Conv1d(128, 256, 3, padding=1)
        self.bn3 = nn.BatchNorm1d(256)
        self.global_pool = nn.AdaptiveMaxPool1d(1)
        self.fc = nn.Linear(256, output_dim)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.pool1(x)
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool2(x)
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.global_pool(x).squeeze(-1)
        x = self.dropout(x)
        x = F.relu(self.fc(x))
        return x

class DenseBranch(nn.Module):
    def __init__(self, input_dim=25, output_dim=128):
        super(DenseBranch, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.bn1 = nn.BatchNorm1d(64)
        self.dropout1 = nn.Dropout(0.3)
        self.fc2 = nn.Linear(64, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.dropout2 = nn.Dropout(0.3)
        self.fc3 = nn.Linear(128, output_dim)

    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout1(x)
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout2(x)
        x = F.relu(self.fc3(x))
        return x

class HybridGenePredictor(nn.Module):
    def __init__(self, num_traditional_features=25):
        super(HybridGenePredictor, self).__init__()
        self.cnn_branch = CNNBranch(128)
        self.dense_branch = DenseBranch(num_traditional_features, 128)
        self.fusion = nn.Sequential(
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 1)  # raw logits, no sigmoid
        )

    def forward(self, sequences, features):
        cnn_out = self.cnn_branch(sequences)
        dense_out = self.dense_branch(features)
        combined = torch.cat([cnn_out, dense_out], dim=1)
        logits = self.fusion(combined)
        return logits.squeeze(-1)  # shape: (batch,)

# ========================
# TRAINING LOOP
# ========================

def train_model(
    model, train_loader, val_loader, device='cuda', 
    num_epochs=30, learning_rate=1e-3, patience=7, mixed_precision=True
):
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.BCEWithLogitsLoss()
    scaler = torch.cuda.amp.GradScaler(enabled=mixed_precision)

    best_val_loss = float('inf')
    best_model_state = None
    history = {'train_loss': [], 'val_loss': []}
    epochs_no_improve = 0

    for epoch in range(num_epochs):
        # -------- TRAIN --------
        model.train()
        train_loss = 0
        for seqs, feats, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            seqs = seqs.to(device)
            feats = feats.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            with torch.cuda.amp.autocast(enabled=mixed_precision):
                outputs = model(seqs, feats)
                loss = criterion(outputs, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            train_loss += loss.item() * seqs.size(0)

        train_loss /= len(train_loader.dataset)
        history['train_loss'].append(train_loss)

        # -------- VALIDATION --------
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for seqs, feats, labels in val_loader:
                seqs = seqs.to(device)
                feats = feats.to(device)
                labels = labels.to(device)
                outputs = model(seqs, feats)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * seqs.size(0)

        val_loss /= len(val_loader.dataset)
        history['val_loss'].append(val_loss)

        print(f"Epoch {epoch+1}: Train Loss={train_loss:.4f}, Val Loss={val_loss:.4f}")

        # -------- EARLY STOPPING --------
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_state = model.state_dict()
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                print(f"Early stopping triggered at epoch {epoch+1}")
                break

    # Load best model
    model.load_state_dict(best_model_state)
    return model, history



In [34]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import torch

def evaluate_model(model, loader, device='cuda', threshold=0.5):
    model.eval()
    preds = []
    targets = []

    with torch.no_grad():
        for seqs, feats, labels in loader:
            seqs, feats = seqs.to(device), feats.to(device)
            labels = labels.cpu().numpy()
            outputs = model(seqs, feats).cpu().numpy()
            predictions = (outputs >= threshold).astype(int)
            
            preds.extend(predictions)
            targets.extend(labels)
    
    acc = accuracy_score(targets, preds)
    prec = precision_score(targets, preds)
    rec = recall_score(targets, preds)
    f1 = f1_score(targets, preds)
    tn, fp, fn, tp = confusion_matrix(targets, preds).ravel()

    print(f"Accuracy:  {acc:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall:    {rec:.4f}")
    print(f"F1 Score:  {f1:.4f}")
    print(f"Confusion Matrix:")
    print(f" TN: {tn} | FP: {fp}")
    print(f" FN: {fn} | TP: {tp}")

# -----------------------------
# 1️⃣ Find the best threshold
# -----------------------------
def find_best_threshold(model, loader, device='cuda'):
    model.eval()
    all_outputs = []
    all_labels = []

    with torch.no_grad():
        for seqs, feats, labels in loader:
            seqs, feats = seqs.to(device), feats.to(device)
            labels = labels.cpu().numpy()
            outputs = model(seqs, feats).cpu().numpy()
            all_outputs.extend(outputs)
            all_labels.extend(labels)

    all_outputs = np.array(all_outputs)
    all_labels = np.array(all_labels)

    best_f1 = 0.0
    best_thresh = 0.5

    thresholds = np.arange(0.1, 0.91, 0.01)
    for t in thresholds:
        preds = (all_outputs >= t).astype(int)
        f1 = f1_score(all_labels, preds)
        if f1 > best_f1:
            best_f1 = f1
            best_thresh = t

    print(f"Best threshold: {best_thresh:.2f} → F1 Score: {best_f1:.4f}")
    return best_thresh, best_f1



In [ ]:

# Example usage


model = HybridGenePredictor(num_traditional_features=25)
trained_model, history = train_model(model, train_loader, val_loader, device='cuda')


evaluate_model(trained_model, val_loader, device='cuda', threshold=0.5)

best_threshold, best_f1 = find_best_threshold(trained_model, val_loader, device='cuda')

project_root = Path.cwd().parent 
save_dir = project_root / "models"
save_dir.mkdir(parents=True, exist_ok=True)

save_path = save_dir / "hybrid_final_filter.pkl"

with open(save_path, "wb") as f:
    pickle.dump({
        "model_state_dict": trained_model.state_dict(),
        "threshold": best_threshold,
        "num_traditional_features": 25
    }, f)

print(f"Saved hybrid model + threshold to {save_path}")


C:\Users\User\AppData\Local\Temp\ipykernel_28216\4034407576.py:92: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=mixed_precision)
Epoch 1/30:   0%|          | 0/1585 [00:00<?, ?it/s]C:\Users\User\AppData\Local\Temp\ipykernel_28216\4034407576.py:109: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=mixed_precision):
Epoch 1/30: 100%|██████████| 1585/1585 [00:29<00:00, 53.14it/s]


Epoch 1: Train Loss=0.4310, Val Loss=0.3483


Epoch 2/30: 100%|██████████| 1585/1585 [00:29<00:00, 54.44it/s]


Epoch 2: Train Loss=0.3675, Val Loss=0.3406


Epoch 3/30: 100%|██████████| 1585/1585 [00:28<00:00, 54.72it/s]


...

Epoch 16: Train Loss=0.2451, Val Loss=0.3250


Epoch 17/30: 100%|██████████| 1585/1585 [00:28<00:00, 56.31it/s]


Epoch 17: Train Loss=0.2388, Val Loss=0.3116


Epoch 18/30: 100%|██████████| 1585/1585 [00:27<00:00, 57.01it/s]


Epoch 18: Train Loss=0.2309, Val Loss=0.3211
Early stopping triggered at epoch 18
Accuracy:  0.8735
Precision: 0.8586
Recall:    0.9425
F1 Score:  0.8986
Confusion Matrix:
 TN: 7939 | FP: 2340
 FN: 867 | TP: 14204
Best threshold: 0.12 → F1 Score: 0.9004
Saved hybrid model + threshold to c:\Users\User\Desktop\Bacterial Gene Prediction & Comparison Project\models\hybrid_final_filter.pkl


In [47]:
evaluate_model(trained_model, val_loader, device='cuda', threshold=0.12)


AUC=0.9278
Confusion matrix:
 [[ 6459  3820]
 [   66 15005]]


(np.float64(0.927814838541356),
 array([[ 6459,  3820],
        [   66, 15005]]))

In [2]:
"""
Test script for HybridGeneFilter on real genome predictions.
"""

import sys
sys.path.insert(0, '..')  

from pathlib import Path
from hybrid_predictor import predict_ncbi_genome
from src.ml_models import HybridGeneFilter

# ============================================================================
# 1️⃣ Get predictions from hybrid_predictor
# ============================================================================
print("="*80)
print("STEP 1: Running hybrid_predictor on E. coli genome")
print("="*80)

pred = predict_ncbi_genome("NC_000913.3", email="your_email@domain.com")

print(f"\n✓ Got {len(pred)} predictions from hybrid_predictor")
print(f"  Sample prediction keys: {list(pred[0].keys())[:10]}")

# Check if 'sequence' key exists
if 'sequence' not in pred[0]:
    print("⚠ WARNING: No 'sequence' key found in predictions!")
    print(f"  Available keys: {list(pred[0].keys())}")
else:
    print(f"✓ Sequence key found: {len(pred[0]['sequence'])} bp")


STEP 1: Running hybrid_predictor on E. coli genome

MODE: NCBI DOWNLOAD
Accession: NC_000913.3
Email: your_email@domain.com

  Target: data\full_dataset\NC_000913.3.fasta
  Fetching sequence...
  Downloaded: 4,774,347 bytes
  ✓ Genome downloaded successfully

RUNNING GENE PREDICTION


MODE: RAW FASTA FILE
Input: data\full_dataset\NC_000913.3.fasta
Output: C:\Users\User\Desktop\Bacterial Gene Prediction & Comparison Project\results\NC_000913.3_predictions.gff
ML filtering: Enabled
ML threshold: 0.1

STEP 1: LOAD FASTA FILE
Loaded: NC_000913.3 Escherichia coli str. K-12 substr. MG1655, complete genome
Genome size: 4,641,652 bp
Sequence loaded: 4,641,652 bp

STEP 2: FIND ALL ORFs
Detecting ORFs and calculating RBS...
Complete: 176,315 ORFs detected with RBS scores
ORFs found: 176,315

STEP 3: CREATE TRAINING SET
Training set: 1,220 ORFs

STEP 4: CREATE INTERGENIC REGIONS
Intergenic regions: 490

STEP 5: BUILD SCORING MODELS
Building traditional scoring models...
  Building codon usage mod

Groups user_genome: 100%|██████████| 6977/6977 [00:00<00:00, 9590.34it/s] 
c:\Users\User\anaconda3\envs\gene_prediction\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(



STEP 10: SELECT BEST START CODONS

Selecting best start for 5,179 groups
  Single option groups: 497
  Multiple option groups: 4,682
Top candidates: 5,179

STEP 11: FINAL FILTERING
Filtered: 4,933 kept, 246 removed
Final predictions: 4,933

STEP 12: WRITE OUTPUT
[+] Wrote 4933 predictions to C:\Users\User\Desktop\Bacterial Gene Prediction & Comparison Project\results\NC_000913.3_predictions.gff

PREDICTION COMPLETE!
Input:  data\full_dataset\NC_000913.3.fasta
Output: C:\Users\User\Desktop\Bacterial Gene Prediction & Comparison Project\results\NC_000913.3_predictions.gff
Size:   4,641,652 bp
Genes:  4,933


NCBI PREDICTION COMPLETE!
Genome:      NC_000913.3
Downloaded:  data\full_dataset\NC_000913.3.fasta
Predictions: results\NC_000913.3_predictions.gff
Total genes: 4,933


✓ Got 4933 predictions from hybrid_predictor
  Sample prediction keys: ['start', 'end', 'genome_start', 'genome_end', 'length', 'frame', 'strand', 'start_codon', 'sequence', 'rbs_score']
✓ Sequence key found: 2463 b

In [ ]:
import torch
import pickle
from pathlib import Path

# Path to your current checkpoint
model_path = Path(r"c:\Users\User\Desktop\Bacterial Gene Prediction & Comparison Project\models\hybrid_final_filter.pkl")

# Load the old checkpoint (dict)
with open(model_path, "rb") as f:
    checkpoint = pickle.load(f)

# Rebuild model architecture
num_feats = checkpoint["num_traditional_features"]
model = HybridGenePredictor(num_traditional_features=num_feats)
model.load_state_dict(checkpoint["model_state_dict"])
model.eval()

# ✅ Overwrite the same file with a true model object
torch.save({
    "model": model,
    "threshold": checkpoint["threshold"]
}, model_path)

print(f"✅ Replaced with full PyTorch model: {model_path}")


✅ Replaced with full PyTorch model: c:\Users\User\Desktop\Bacterial Gene Prediction & Comparison Project\models\hybrid_final_filter.pkl


In [29]:
# ============================================================================
# 2️⃣ Load and apply HybridGeneFilter model
# ============================================================================
print("\n" + "="*80)
print("STEP 2: ML HYBRID FILTERING")
print("="*80)

try:
    classifier = HybridGeneFilter()
    try:
        model_path = Path(__file__).parent / 'models' / 'hybrid_final_filter.pkl'
    except NameError:
    # Fallback when running in Jupyter
        model_path = Path.cwd().parent / 'models' / 'hybrid_final_filter.pkl'
    
    if model_path.exists():
        classifier.load(str(model_path))
        
        # Filter predictions
        filtered_pred = classifier.filter_candidates(
            candidates=pred,
            genome_id="NC_000913.3",
            threshold=0.12
        )
        
        print(f"\n✓ Filtered predictions complete!")
        print(f"  Before filtering: {len(pred):,} predictions")
        print(f"  After filtering:  {len(filtered_pred):,} predictions")
        print(f"  Kept: {100*len(filtered_pred)/len(pred):.1f}%")
        
    else:
        print(f"[!] Model not found at: {model_path}")
        print(f"[!] Skipping ML filtering...")
        filtered_pred = pred
        
except Exception as e:
    print(f"[!] ML error: {e}")
    print(f"[!] Skipping ML filtering...")
    import traceback
    traceback.print_exc()
    filtered_pred = pred

# ============================================================================
# 3️⃣ Show some statistics
# ============================================================================
print("\n" + "="*80)
print("STEP 3: Statistics")
print("="*80)

if filtered_pred and 'hybrid_prob' in filtered_pred[0]:
    # Get probability distribution
    import numpy as np
    probs = [p.get('hybrid_prob', 0) for p in filtered_pred]
    
    print(f"\nHybrid probability statistics:")
    print(f"  Min:    {np.min(probs):.4f}")
    print(f"  Max:    {np.max(probs):.4f}")
    print(f"  Mean:   {np.mean(probs):.4f}")
    print(f"  Median: {np.median(probs):.4f}")
    
    # Show top 5 predictions
    print(f"\nTop 5 predictions by hybrid_prob:")
    sorted_pred = sorted(filtered_pred, key=lambda x: x.get('hybrid_prob', 0), reverse=True)
    for i, p in enumerate(sorted_pred[:5], 1):
        print(f"  {i}. {p.get('gene_id', 'unknown'):20s} "
              f"prob={p.get('hybrid_prob', 0):.4f} "
              f"len={p.get('length', 0):4d} "
              f"combined={p.get('combined_score', 0):.3f}")
    
    # Show bottom 5 predictions (just above threshold)
    print(f"\nBottom 5 predictions (just above threshold):")
    for i, p in enumerate(sorted_pred[-5:], 1):
        print(f"  {i}. {p.get('gene_id', 'unknown'):20s} "
              f"prob={p.get('hybrid_prob', 0):.4f} "
              f"len={p.get('length', 0):4d} "
              f"combined={p.get('combined_score', 0):.3f}")
else:
    print("\n✓ Basic statistics:")
    print(f"  Total predictions: {len(filtered_pred):,}")
    if filtered_pred:
        print(f"  Sample keys: {list(filtered_pred[0].keys())[:10]}")

# ============================================================================
# 4️⃣ Save results to GFF
# ============================================================================
print("\n" + "="*80)
print("STEP 4: Save results")
print("="*80)

def write_gff(predictions: list, output_path: str, sequence_id: str = "sequence"):
    """Write predictions to GFF3 format."""
    with open(output_path, 'w') as f:
        f.write("##gff-version 3\n")
        for i, pred in enumerate(predictions, 1):
            start = pred.get('genome_start', pred.get('start'))
            end = pred.get('genome_end', pred.get('end'))
            strand = '+' if pred.get('strand') == 'forward' else '-'
            score = pred.get('combined_score', 0.0)
            rbs = pred.get('rbs_score', 0.0)
            hybrid_prob = pred.get('hybrid_prob', 'NA')
            
            # Add hybrid_prob to attributes if available
            if hybrid_prob != 'NA':
                attrs = f"ID=gene_{i};rbs_score={rbs:.2f};combined_score={score:.2f};hybrid_prob={hybrid_prob:.4f}"
            else:
                attrs = f"ID=gene_{i};rbs_score={rbs:.2f};combined_score={score:.2f}"
            
            f.write(f"{sequence_id}\tHybridPredictor\tCDS\t{start}\t{end}\t{score:.3f}\t{strand}\t0\t{attrs}\n")
    print(f"[+] Wrote {len(predictions)} predictions to {output_path}")

# Create results directory
# Handle both script and notebook execution
try:
    results_dir = Path(__file__).resolve().parent / 'results'
except NameError:
    # Running in Jupyter notebook
    results_dir = Path.cwd() / 'results'

results_dir.mkdir(exist_ok=True)

# Save filtered predictions
filename = 'NC_000913.3_filtered_predictions.gff'
output_path = str(results_dir / filename)
write_gff(filtered_pred, output_path, sequence_id="NC_000913.3")

# Also save original predictions for comparison
filename_original = 'NC_000913.3_original_predictions.gff'
output_path_original = str(results_dir / filename_original)
write_gff(pred, output_path_original, sequence_id="NC_000913.3")

print(f"\n✓ Saved filtered predictions: {output_path}")
print(f"✓ Saved original predictions: {output_path_original}")

print("\n" + "="*80)
print("✓ TEST COMPLETE")
print("="*80)


STEP 2: ML HYBRID FILTERING
[!] ML error: persistent IDs in protocol 0 must be ASCII strings
[!] Skipping ML filtering...

STEP 3: Statistics

✓ Basic statistics:
  Total predictions: 4,933
  Sample keys: ['start', 'end', 'genome_start', 'genome_end', 'length', 'frame', 'strand', 'start_codon', 'sequence', 'rbs_score']

STEP 4: Save results
[+] Wrote 4933 predictions to c:\Users\User\Desktop\Bacterial Gene Prediction & Comparison Project\notebooks\results\NC_000913.3_filtered_predictions.gff
[+] Wrote 4933 predictions to c:\Users\User\Desktop\Bacterial Gene Prediction & Comparison Project\notebooks\results\NC_000913.3_original_predictions.gff

✓ Saved filtered predictions: c:\Users\User\Desktop\Bacterial Gene Prediction & Comparison Project\notebooks\results\NC_000913.3_filtered_predictions.gff
✓ Saved original predictions: c:\Users\User\Desktop\Bacterial Gene Prediction & Comparison Project\notebooks\results\NC_000913.3_original_predictions.gff

✓ TEST COMPLETE


Traceback (most recent call last):
  File "c:\Users\User\anaconda3\envs\gene_prediction\lib\pickle.py", line 1244, in load_persid
    pid = self.readline()[:-1].decode("ascii")
UnicodeDecodeError: 'ascii' codec can't decode byte 0x80 in position 63: ordinal not in range(128)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\User\AppData\Local\Temp\ipykernel_29520\125135764.py", line 17, in <module>
    classifier.load(str(model_path))
  File "c:\Users\User\Desktop\Bacterial Gene Prediction & Comparison Project\notebooks\..\src\ml_models.py", line 307, in load
    # Load the checkpoint (contains full model + threshold)
  File "c:\Users\User\anaconda3\envs\gene_prediction\lib\site-packages\joblib\numpy_pickle.py", line 749, in load
    obj = _unpickle(
  File "c:\Users\User\anaconda3\envs\gene_prediction\lib\site-packages\joblib\numpy_pickle.py", line 626, in _unpickle
    obj = unpickler.load()
  File "c:\Users\User